Notebook for cross-matching the CAMIRA S19A public and internal catalogs.

In [63]:
## Import packages
import numpy as np
from scipy.optimize import minimize
from scipy import integrate

from astropy.io import fits, ascii
from astropy import units as u

from astropy.coordinates import angles
from astropy.coordinates import Angle
from astropy.coordinates import SkyCoord
from astropy import constants as const
from astropy.cosmology import LambdaCDM
from astropy.table import Table
from astropy.visualization import quantity_support
quantity_support()

import matplotlib.pyplot as plt

# Load in both catalogs

**Internal:** Each row has 6 values: RA (degrees), Dec (degrees), z_cl, N_mem, logMs, and z_bcg; in that order. z_bcg = -1.0 indicates that the cluster has no identified BCG.

**Public:** Each row has 7 values: ID, name, RA, Dec, z_cl, N_mem, and z_bcg.

In [7]:
# load internal catalog + make richness cut
internal_cat_orig = np.loadtxt('camira_s19a_wide_sm_v1.txt', usecols=(0,1,2,3,4,5)) 
internal_cat_IDs_orig = np.loadtxt('camira_s19a_wide_sm_v1.txt', dtype='str', usecols=(6)) 

good_richness = internal_cat_orig[:,3] > 15
internal_cat = internal_cat_orig[good_richness]
internal_cat_IDs = internal_cat_IDs_orig[good_richness]

# load public catalog + make richness cut
public_cat_orig = np.loadtxt('camira_s19a_wide_sm_PUBLIC.txt', usecols=(2,3,4,5,6)) 
public_cat_IDs_orig = np.loadtxt('camira_s19a_wide_sm_PUBLIC.txt', dtype='str', usecols=(1)) 

good_richness = public_cat_orig[:,3] > 15
public_cat = public_cat_orig[good_richness]
public_cat_IDs = public_cat_IDs_orig[good_richness]

Check that the public and internal catalogs have the same number of clusters (5860).

In [ ]:
len(public_cat)

# Create SkyCoords

In [13]:
## SkyCoord objects for both catalogs
internal_coords = SkyCoord(ra=internal_cat[:,0] * u.degree, 
                      dec=internal_cat[:,1] * u.degree, 
                      frame='icrs')
public_coords = SkyCoord(ra=public_cat[:,0] * u.degree, 
                      dec=public_cat[:,1] * u.degree, 
                      frame='icrs')

# convert coords to (-pi, pi) radians
internal_ra = internal_coords.ra.wrap_at(180 * u.deg).radian
internal_dec = internal_coords.dec.radian
public_ra = public_coords.ra.wrap_at(180 * u.deg).radian
public_dec = public_coords.dec.radian


Good news, it looks like the clusters in the public vs. internal catalogs exactly match up (i.e., the first cluster in each list is the same, and so on).

In [42]:
ind = 3
np.sum(np.abs(internal_cat[:, ind] - public_cat[:,ind]))

0.0

In [23]:
public_cat[:,0]

array([2.00800000e-03, 1.12666000e-01, 1.97932000e-01, ...,
       3.59889826e+02, 3.59892565e+02, 3.59893792e+02])

Get indices of HSC clusters that are in the cross-match.

In [48]:
## idx array (len = 4195)
idx = [5843, 5843, 5843, 5843, 3, 5843, 0, 229, 5843, 229, 5843, 8, 5843, 5853, 5853, 5843, 29, 162, 18, 5853, 5843, 5843, 5843, 5843, 5853, 5843, 5843, 78, 42, 5843, 78, 5843, 56, 47, 162, 5843, 5853, 71, 5843, 5843, 5843, 79, 78, 75, 79, 5843, 5843, 5843, 78, 5843, 87, 78, 79, 5843, 103, 89, 5843, 78, 89, 78, 5843, 78, 78, 89, 79, 5843, 5843, 79, 5843, 5843, 78, 5843, 5843, 162, 5843, 89, 5843, 162, 89, 89, 5843, 5853, 162, 105, 107, 98, 5843, 78, 5853, 5843, 79, 5843, 162, 105, 5843, 89, 98, 162, 89, 5843, 89, 162, 5853, 5853, 5843, 105, 108, 78, 5843, 78, 162, 5853, 108, 5843, 89, 229, 5853, 107, 107, 78, 107, 107, 162, 89, 89, 89, 162, 5853, 108, 162, 162, 162, 162, 78, 89, 78, 89, 89, 107, 89, 5853, 162, 89, 89, 108, 177, 98, 89, 89, 89, 162, 229, 89, 105, 89, 162, 109, 229, 78, 107, 107, 89, 89, 89, 229, 89, 162, 78, 89, 105, 162, 162, 89, 162, 107, 162, 89, 89, 89, 162, 107, 162, 98, 162, 105, 89, 89, 108, 162, 105, 89, 89, 89, 89, 98, 98, 89, 109, 78, 78, 107, 108, 89, 109, 78, 89, 89, 89, 78, 108, 98, 98, 107, 89, 229, 109, 105, 162, 107, 162, 162, 89, 107, 89, 89, 162, 229, 89, 107, 109, 107, 162, 89, 89, 162, 89, 107, 98, 98, 105, 229, 89, 108, 108, 162, 109, 229, 108, 109, 89, 78, 162, 162, 105, 229, 162, 162, 162, 162, 229, 105, 162, 177, 105, 89, 162, 107, 162, 89, 109, 105, 89, 89, 162, 109, 107, 229, 98, 98, 162, 108, 109, 109, 162, 162, 108, 108, 162, 89, 108, 177, 162, 105, 162, 98, 162, 105, 162, 89, 229, 78, 105, 177, 108, 177, 229, 105, 162, 229, 229, 98, 89, 152, 229, 107, 105, 98, 229, 107, 109, 105, 109, 229, 177, 229, 229, 107, 105, 107, 162, 107, 229, 107, 98, 105, 162, 162, 108, 152, 162, 98, 109, 162, 229, 107, 109, 177, 109, 229, 109, 229, 108, 107, 98, 107, 108, 107, 107, 229, 109, 98, 108, 229, 109, 229, 98, 109, 108, 162, 109, 111, 109, 152, 107, 109, 162, 229, 109, 162, 109, 109, 229, 229, 111, 162, 111, 229, 107, 107, 109, 162, 107, 107, 162, 229, 110, 107, 109, 107, 130, 162, 162, 162, 110, 162, 111, 162, 229, 229, 229, 229, 110, 110, 229, 162, 229, 107, 229, 229, 229, 124, 229, 130, 229, 229, 162, 111, 162, 110, 229, 110, 229, 229, 131, 111, 229, 229, 162, 110, 229, 177, 131, 229, 229, 124, 162, 111, 110, 110, 131, 229, 124, 152, 131, 124, 162, 229, 131, 177, 162, 229, 229, 124, 229, 162, 229, 162, 131, 162, 124, 162, 110, 229, 229, 229, 131, 229, 131, 131, 162, 229, 130, 162, 229, 162, 162, 229, 229, 130, 229, 131, 162, 229, 229, 229, 110, 229, 131, 111, 131, 229, 110, 110, 111, 229, 111, 156, 162, 110, 156, 162, 229, 162, 229, 131, 229, 229, 229, 124, 130, 124, 177, 111, 131, 131, 229, 131, 131, 229, 229, 111, 111, 162, 111, 229, 156, 229, 110, 111, 229, 229, 229, 111, 229, 162, 110, 111, 111, 156, 130, 111, 130, 162, 162, 156, 111, 131, 110, 152, 124, 131, 152, 131, 229, 152, 229, 229, 156, 111, 229, 229, 124, 162, 131, 229, 152, 156, 162, 131, 131, 156, 162, 110, 229, 156, 229, 162, 229, 229, 131, 156, 177, 229, 130, 130, 156, 110, 131, 229, 111, 162, 229, 229, 131, 152, 131, 229, 124, 130, 162, 162, 111, 177, 111, 124, 131, 131, 111, 130, 229, 110, 162, 229, 229, 111, 229, 156, 162, 177, 111, 229, 229, 152, 229, 229, 229, 229, 130, 229, 229, 229, 229, 229, 156, 130, 130, 229, 229, 229, 156, 110, 229, 229, 162, 156, 110, 152, 229, 156, 229, 111, 229, 229, 229, 229, 152, 131, 229, 111, 229, 119, 229, 229, 156, 124, 119, 162, 229, 229, 162, 229, 229, 229, 229, 229, 229, 131, 156, 229, 229, 229, 156, 111, 229, 229, 111, 162, 229, 131, 229, 229, 229, 229, 229, 156, 119, 229, 110, 156, 229, 110, 162, 229, 125, 162, 124, 229, 162, 229, 229, 156, 141, 156, 162, 229, 229, 119, 156, 229, 229, 156, 113, 156, 229, 229, 152, 229, 229, 229, 229, 136, 229, 229, 156, 229, 156, 146, 148, 229, 229, 229, 150, 156, 229, 229, 229, 229, 229, 229, 156, 229, 229, 229, 229, 229, 156, 156, 229, 156, 184, 179, 156, 229, 229, 229, 229, 229, 213, 229, 217, 229, 229, 156, 213, 156, 229, 226, 229, 229, 213, 229, 156, 240, 244, 229, 253, 229, 229, 229, 229, 229, 229, 279, 229, 271, 229, 275, 213, 268, 285, 229, 213, 229, 340, 229, 312, 229, 229, 340, 306, 340, 308, 229, 229, 312, 229, 229, 312, 340, 229, 312, 229, 343, 312, 229, 229, 340, 229, 356, 340, 358, 340, 363, 229, 312, 340, 312, 340, 385, 312, 340, 312, 375, 340, 340, 340, 340, 312, 390, 340, 312, 340, 340, 340, 340, 340, 414, 417, 340, 340, 312, 340, 479, 479, 467, 432, 312, 455, 455, 312, 479, 479, 479, 479, 479, 479, 479, 479, 479, 479, 479, 479, 459, 467, 479, 479, 479, 312, 479, 472, 479, 467, 479, 479, 467, 479, 312, 467, 467, 479, 479, 479, 312, 479, 479, 479, 467, 467, 467, 479, 479, 312, 479, 467, 501, 479, 479, 312, 479, 467, 479, 514, 517, 312, 467, 479, 479, 479, 524, 479, 479, 479, 467, 565, 479, 479, 479, 479, 479, 467, 524, 565, 479, 312, 479, 479, 479, 571, 479, 479, 479, 479, 479, 479, 467, 467, 608, 479, 608, 479, 608, 479, 594, 479, 608, 608, 608, 591, 651, 651, 651, 614, 467, 467, 608, 651, 651, 651, 591, 651, 651, 651, 651, 651, 651, 651, 467, 638, 479, 651, 627, 645, 651, 651, 467, 627, 627, 651, 467, 651, 651, 651, 627, 627, 627, 651, 651, 651, 660, 627, 651, 651, 651, 651, 651, 651, 651, 653, 666, 651, 651, 651, 651, 467, 670, 651, 637, 651, 651, 651, 651, 627, 651, 651, 467, 679, 680, 651, 651, 651, 651, 651, 682, 651, 651, 651, 627, 651, 651, 651, 651, 651, 651, 686, 651, 651, 651, 679, 651, 679, 627, 467, 651, 651, 627, 679, 651, 651, 685, 651, 651, 686, 651, 627, 651, 679, 685, 627, 686, 651, 679, 651, 651, 651, 651, 680, 680, 680, 651, 627, 627, 651, 651, 651, 651, 651, 627, 627, 679, 679, 686, 651, 651, 651, 651, 679, 679, 651, 680, 679, 651, 651, 651, 679, 651, 651, 651, 686, 651, 679, 686, 679, 686, 651, 651, 651, 680, 679, 679, 686, 680, 651, 651, 651, 627, 651, 679, 651, 651, 651, 679, 651, 651, 651, 680, 651, 651, 651, 679, 651, 651, 651, 651, 651, 651, 679, 651, 651, 651, 684, 651, 651, 680, 651, 680, 627, 651, 651, 651, 651, 651, 651, 679, 680, 651, 686, 651, 651, 651, 651, 651, 651, 679, 651, 651, 679, 651, 651, 679, 680, 684, 651, 684, 651, 651, 679, 680, 679, 651, 651, 686, 686, 679, 651, 680, 651, 651, 680, 680, 686, 651, 680, 679, 651, 684, 627, 651, 651, 651, 651, 651, 651, 651, 651, 679, 679, 679, 651, 651, 680, 680, 651, 651, 684, 651, 680, 651, 651, 651, 651, 651, 680, 651, 679, 686, 651, 651, 686, 679, 651, 679, 651, 651, 651, 651, 686, 680, 680, 684, 679, 651, 651, 679, 651, 651, 679, 651, 651, 679, 680, 686, 651, 679, 680, 684, 680, 680, 680, 679, 651, 651, 651, 651, 651, 651, 651, 680, 686, 680, 680, 680, 679, 686, 651, 686, 651, 680, 651, 679, 686, 686, 686, 651, 651, 651, 651, 680, 651, 684, 651, 651, 686, 651, 679, 651, 651, 686, 686, 680, 651, 651, 651, 680, 680, 651, 680, 680, 651, 680, 684, 680, 651, 684, 651, 651, 680, 651, 686, 651, 651, 651, 680, 651, 651, 651, 680, 651, 680, 686, 680, 680, 684, 684, 651, 680, 680, 651, 651, 680, 680, 651, 651, 651, 651, 684, 679, 680, 651, 651, 684, 651, 680, 680, 680, 680, 684, 651, 680, 651, 680, 680, 651, 651, 651, 651, 679, 651, 651, 651, 680, 651, 651, 680, 680, 680, 680, 680, 651, 651, 680, 651, 684, 651, 680, 651, 651, 679, 680, 686, 651, 651, 651, 651, 651, 651, 686, 651, 651, 684, 651, 680, 680, 680, 680, 651, 651, 680, 651, 651, 684, 651, 680, 680, 680, 651, 651, 680, 680, 686, 686, 680, 680, 651, 679, 680, 651, 679, 680, 651, 680, 680, 680, 651, 651, 686, 686, 680, 651, 680, 680, 680, 680, 651, 651, 651, 679, 679, 680, 651, 651, 651, 680, 680, 679, 680, 680, 680, 680, 651, 680, 680, 680, 651, 680, 684, 680, 684, 651, 651, 680, 680, 684, 684, 651, 684, 651, 680, 680, 680, 680, 680, 680, 680, 686, 680, 651, 679, 680, 651, 680, 679, 680, 680, 684, 680, 651, 680, 651, 686, 651, 684, 680, 651, 651, 651, 680, 680, 680, 651, 680, 680, 651, 651, 686, 686, 651, 680, 686, 680, 651, 680, 684, 680, 680, 684, 680, 680, 680, 680, 684, 680, 680, 680, 680, 651, 680, 680, 680, 680, 680, 680, 651, 680, 684, 651, 686, 680, 680, 686, 680, 680, 680, 680, 686, 680, 684, 651, 680, 680, 684, 686, 680, 651, 686, 686, 680, 680, 651, 684, 686, 680, 686, 651, 680, 680, 680, 651, 680, 680, 686, 680, 679, 684, 680, 686, 680, 680, 680, 680, 651, 686, 686, 679, 680, 680, 651, 651, 651, 684, 686, 680, 680, 680, 684, 684, 680, 680, 679, 684, 651, 686, 680, 680, 651, 680, 680, 680, 684, 686, 651, 680, 680, 680, 680, 680, 680, 680, 686, 680, 684, 684, 680, 680, 680, 686, 686, 684, 684, 680, 686, 684, 680, 680, 686, 684, 651, 686, 680, 684, 680, 680, 680, 686, 680, 684, 680, 686, 680, 680, 680, 680, 651, 680, 680, 680, 680, 680, 680, 680, 684, 680, 680, 686, 680, 684, 680, 680, 680, 680, 680, 680, 680, 680, 680, 680, 686, 684, 680, 680, 680, 680, 680, 680, 684, 684, 680, 680, 680, 680, 651, 680, 680, 680, 686, 680, 680, 680, 680, 680, 680, 680, 680, 680, 680, 680, 686, 684, 680, 680, 680, 680, 684, 680, 680, 680, 680, 680, 680, 680, 680, 680, 651, 680, 680, 680, 680, 680, 680, 680, 680, 680, 680, 680, 680, 680, 680, 680, 680, 680, 680, 680, 680, 684, 680, 680, 680, 680, 680, 680, 680, 680, 680, 680, 680, 680, 680, 684, 684, 680, 680, 680, 680, 693, 693, 680, 680, 680, 693, 680, 680, 693, 680, 680, 680, 680, 680, 716, 680, 716, 693, 693, 680, 716, 680, 716, 693, 680, 693, 693, 693, 716, 680, 680, 716, 716, 693, 693, 716, 693, 693, 693, 721, 716, 721, 716, 721, 693, 721, 716, 721, 693, 721, 721, 716, 716, 716, 716, 721, 716, 716, 716, 721, 716, 721, 721, 693, 721, 721, 721, 716, 721, 721, 716, 721, 721, 721, 716, 721, 721, 721, 721, 721, 721, 721, 721, 721, 721, 721, 721, 721, 721, 721, 721, 721, 721, 721, 721, 721, 721, 721, 721, 721, 699, 699, 699, 699, 699, 699, 699, 699, 699, 699, 699, 699, 699, 699, 699, 699, 699, 699, 699, 699, 699, 729, 729, 729, 687, 729, 699, 699, 699, 687, 699, 687, 687, 729, 792, 792, 729, 729, 690, 687, 687, 699, 687, 695, 687, 687, 729, 729, 689, 729, 729, 792, 729, 729, 792, 729, 687, 694, 828, 758, 770, 828, 782, 770, 828, 828, 805, 828, 828, 828, 828, 828, 839, 828, 834, 828, 876, 828, 880, 828, 888, 902, 828, 928, 929, 970, 942, 932, 970, 970, 970, 970, 970, 981, 970, 997, 970, 970, 970, 1011, 970, 1014, 1058, 1020, 1038, 1058, 1033, 1042, 1058, 1066, 1058, 1058, 1058, 1080, 1090, 1058, 1058, 1085, 1058, 1101, 1108, 1058, 1116, 1125, 1058, 1139, 1131, 1156, 1058, 1156, 1058, 1180, 1182, 1246, 1246, 1246, 1206, 1246, 1246, 1246, 1246, 1246, 1246, 1334, 1247, 1334, 1277, 1298, 1300, 1334, 1311, 1334, 1291, 1375, 1334, 1334, 1343, 1375, 1376, 1365, 1438, 1396, 1438, 1438, 1413, 1410, 1438, 1438, 1438, 1438, 1438, 1438, 1438, 1438, 1484, 1438, 1438, 1556, 1484, 1438, 1555, 1438, 1484, 1438, 1438, 1617, 1606, 1606, 1593, 1617, 1593, 1595, 1438, 1438, 1438, 1650, 1658, 1650, 1650, 1438, 1650, 1650, 1694, 1650, 1650, 1763, 1760, 1761, 1650, 1763, 1763, 1650, 1650, 1763, 1763, 1763, 1763, 1803, 1763, 1763, 1806, 1896, 1896, 1896, 1896, 1844, 1896, 1896, 1834, 1830, 1896, 1842, 1896, 1896, 1896, 1896, 1896, 1896, 1896, 1896, 1945, 1896, 1896, 1896, 1942, 1896, 1919, 1896, 1955, 1985, 2006, 1896, 1896, 2006, 2006, 2006, 2002, 2006, 2006, 2006, 2006, 2006, 2006, 1986, 1986, 2006, 2020, 2006, 2028, 2030, 2033, 2033, 2006, 2037, 2006, 2006, 2006, 2006, 2006, 2006, 2006, 2006, 2006, 2051, 2064, 2067, 2006, 2006, 2006, 2006, 2033, 2006, 2065, 2033, 2071, 2033, 2033, 2012, 2107, 2012, 2012, 2006, 2006, 2033, 2099, 2099, 2100, 2107, 2033, 2115, 2012, 2012, 2126, 2012, 2107, 2006, 2107, 2006, 2135, 2006, 2033, 2012, 2012, 2012, 2156, 2156, 2145, 2156, 2033, 2033, 2148, 2006, 2152, 2156, 2006, 2161, 2156, 2156, 2156, 2012, 2012, 2156, 2179, 2179, 2012, 2199, 2156, 2199, 2211, 2213, 2206, 2206, 2206, 2246, 2206, 2206, 2228, 2242, 2242, 2243, 2246, 2251, 2300, 2251, 2251, 2251, 2251, 2251, 2300, 2254, 2251, 2300, 2287, 2300, 2251, 2287, 2287, 2300, 2300, 2271, 2300, 2278, 2559, 2300, 2283, 2300, 2300, 2300, 2300, 2300, 2300, 2300, 2338, 2338, 2300, 2325, 2338, 2338, 2337, 2341, 2338, 2338, 2559, 2338, 2559, 2338, 2338, 2338, 2559, 2338, 2338, 2338, 2559, 2353, 2352, 2352, 2559, 2355, 2559, 2559, 2559, 2559, 2338, 2559, 2352, 2559, 2559, 2559, 2352, 2559, 2559, 2352, 2390, 2559, 2559, 2559, 2559, 2396, 2385, 2559, 2380, 2397, 2399, 2559, 2559, 2559, 2559, 2559, 2559, 2559, 2559, 2559, 2559, 2448, 2559, 2559, 2559, 2463, 2472, 2559, 2559, 2520, 2489, 2483, 2559, 2559, 2559, 2559, 2559, 2559, 2559, 2496, 2520, 2559, 2559, 2559, 2508, 2559, 2559, 2643, 2525, 2535, 2559, 2643, 2548, 2554, 2643, 2643, 2643, 2643, 2643, 2620, 2643, 2621, 2643, 2643, 2643, 2643, 2638, 2643, 2643, 2643, 2643, 2643, 2643, 2643, 2643, 2688, 2643, 2643, 2643, 2643, 2643, 2643, 2643, 2689, 2689, 2643, 2643, 2643, 2745, 2643, 2747, 2799, 2799, 2799, 2799, 2799, 2763, 2799, 2799, 2799, 2799, 2779, 2799, 2799, 2799, 2799, 2799, 2799, 2799, 2834, 2799, 2838, 2799, 2799, 2850, 2799, 2850, 2869, 2834, 2799, 2834, 2834, 2834, 2834, 2834, 2799, 2834, 2904, 2834, 2896, 2834, 2834, 2904, 2913, 2834, 2834, 2834, 2834, 2834, 2834, 2834, 2924, 2834, 2834, 2952, 2924, 2834, 2834, 2834, 2899, 2974, 2970, 2834, 2834, 2834, 3031, 2834, 2834, 2834, 3026, 3031, 3031, 3031, 3063, 2834, 3031, 2834, 2834, 3031, 2834, 2834, 3075, 3083, 3090, 3090, 3090, 3083, 3090, 3090, 3090, 3090, 3090, 3090, 3112, 3090, 3128, 3090, 3130, 3090, 3090, 3136, 3136, 3090, 3136, 3136, 3162, 3090, 3136, 3136, 3136, 3136, 3090, 3090, 3136, 3136, 3136, 3136, 3136, 3136, 3136, 3136, 3136, 3136, 3136, 3191, 3191, 3136, 3136, 3136, 3136, 3136, 3191, 3136, 3136, 3136, 3136, 3136, 3136, 3136, 3136, 3136, 3136, 3136, 3211, 3136, 3136, 3136, 3136, 3136, 3136, 3136, 3136, 3265, 3233, 3136, 3136, 3136, 3265, 3136, 3136, 3136, 3136, 3136, 3136, 3136, 3136, 3267, 3136, 3136, 3241, 3241, 3241, 3265, 3136, 3241, 3267, 3267, 3267, 3267, 3136, 3136, 3279, 3347, 3283, 3347, 3136, 3136, 3136, 3347, 3347, 3136, 3136, 3315, 3347, 3347, 3347, 3347, 3136, 3347, 3136, 3347, 3347, 3347, 3347, 3347, 3136, 3360, 3334, 3335, 3346, 3347, 3360, 3347, 3347, 3347, 3347, 3347, 3347, 3347, 3347, 3360, 3347, 3347, 3347, 3360, 3347, 3347, 3347, 3378, 3347, 3347, 3347, 3347, 3347, 3378, 3390, 3378, 3347, 3401, 3378, 3378, 3411, 3378, 3378, 3378, 3401, 3378, 3411, 3426, 3426, 3426, 3410, 3426, 3426, 3426, 3426, 3426, 3426, 3426, 3426, 3429, 3426, 3426, 3426, 3426, 3429, 3429, 3426, 3426, 3426, 3426, 3426, 3429, 3429, 3439, 3429, 3429, 3429, 3429, 3429, 3439, 3438, 3429, 3429, 3429, 3438, 3429, 3439, 3438, 3438, 3443, 3429, 3438, 3429, 3429, 3438, 3429, 3429, 3429, 3429, 3438, 3443, 3443, 3429, 3438, 3438, 3444, 3450, 3444, 3450, 3450, 3450, 3450, 3450, 3450, 3445, 3450, 3450, 3450, 3450, 3450, 3447, 3447, 3450, 3450, 3450, 3450, 3450, 3450, 3450, 3450, 3447, 3450, 3450, 3450, 3450, 3450, 3450, 3450, 3450, 3450, 3450, 3462, 3450, 3450, 3461, 3450, 3450, 3455, 3450, 3450, 3450, 3445, 3450, 3450, 3450, 3462, 3449, 3450, 3462, 3450, 3450, 3450, 3450, 3462, 3462, 3450, 3450, 3450, 3462, 3450, 3450, 3450, 3461, 3462, 3450, 3450, 3450, 3450, 3482, 3450, 3450, 3496, 3450, 3450, 3450, 3450, 3450, 3450, 3496, 3450, 3450, 3450, 3450, 3450, 3450, 3482, 3482, 3450, 3552, 3552, 3552, 3552, 3522, 3552, 3552, 3552, 3552, 3552, 3552, 3544, 3552, 3552, 3552, 3552, 3552, 3539, 3560, 3547, 3560, 3560, 3560, 3560, 3560, 3560, 3560, 3560, 3546, 3560, 3560, 3605, 3605, 3605, 3605, 3605, 3605, 3605, 3605, 3605, 3599, 3605, 3605, 3605, 3605, 3597, 3603, 3605, 3605, 3605, 3605, 3605, 3605, 3605, 3605, 3605, 3607, 3611, 3605, 3605, 3605, 3605, 3605, 3605, 3605, 3605, 3652, 3652, 3652, 3652, 3652, 3677, 3677, 3652, 3652, 3696, 3742, 3742, 3742, 3742, 3742, 3742, 3742, 3742, 3742, 3742, 3742, 3742, 3742, 3742, 3742, 3742, 3742, 3742, 3742, 3757, 3742, 3742, 3761, 3763, 3742, 3769, 3742, 3742, 3742, 3782, 3782, 3742, 3742, 3782, 3742, 3799, 3811, 3807, 3838, 3811, 3861, 3861, 3811, 3838, 3861, 3838, 3823, 3861, 3861, 3861, 3861, 3861, 3861, 3861, 3861, 3861, 3861, 3892, 3892, 3892, 3892, 3892, 3892, 3892, 3892, 3892, 3935, 3892, 4001, 4001, 4001, 4003, 4001, 4001, 4003, 4003, 4003, 4003, 4003, 3992, 4003, 4003, 4003, 4003, 4003, 4003, 4003, 4013, 4003, 4003, 4003, 4035, 4003, 4003, 4054, 4003, 4039, 4054, 4039, 4054, 4054, 4076, 4076, 4039, 4076, 4076, 4076, 4076, 4076, 4076, 4117, 4076, 4076, 4076, 4076, 4117, 4076, 4109, 4076, 4076, 4113, 4076, 4076, 4076, 4160, 4141, 4160, 4160, 4128, 4160, 4160, 4169, 4160, 4160, 4160, 4160, 4160, 4169, 4160, 4169, 4169, 4169, 4169, 4195, 4167, 4169, 4169, 4184, 4169, 4169, 4169, 4209, 4169, 4169, 4169, 4214, 4169, 4169, 4169, 4169, 4184, 4184, 4226, 4184, 4184, 4184, 4184, 4184, 4184, 4169, 4184, 4209, 4184, 4184, 4209, 4184, 4184, 4226, 4209, 4209, 4184, 4184, 4209, 4184, 4226, 4184, 4184, 4184, 4184, 4184, 4184, 4209, 4232, 4209, 4184, 4184, 4209, 4209, 4184, 4184, 4232, 4184, 4232, 4184, 4184, 4184, 4184, 4184, 4184, 4184, 4209, 4184, 4209, 4184, 4209, 4232, 4232, 4209, 4184, 4209, 4184, 4209, 4209, 4184, 4209, 4184, 4184, 4209, 4209, 4231, 4184, 4184, 4209, 4209, 4232, 4184, 4209, 4184, 4209, 4209, 4209, 4184, 4232, 4209, 4184, 4209, 4209, 4209, 4184, 4184, 4209, 4209, 4184, 4232, 4209, 4209, 4209, 4184, 4184, 4209, 4232, 4209, 4232, 4209, 4232, 4209, 4232, 4209, 4232, 4209, 4232, 4232, 4209, 4209, 4209, 4209, 4232, 4232, 4209, 4209, 4231, 4209, 4209, 4209, 4232, 4232, 4209, 4231, 4209, 4232, 4184, 4231, 4209, 4232, 4209, 4184, 4232, 4209, 4231, 4231, 4232, 4209, 4209, 4232, 4209, 4209, 4232, 4209, 4232, 4231, 4209, 4209, 4209, 4232, 4209, 4209, 4209, 4232, 4209, 4209, 4232, 4232, 4232, 4232, 4209, 4209, 4231, 4209, 4209, 4461, 4232, 4232, 4209, 4232, 4209, 4232, 4209, 4209, 4209, 4209, 4209, 4232, 4232, 4209, 4209, 4209, 4209, 4209, 4232, 4209, 4209, 4232, 4232, 4209, 4209, 4232, 4232, 4209, 4209, 4209, 4232, 4209, 4232, 4209, 4231, 4209, 4209, 4232, 4209, 4209, 4209, 4232, 4209, 4209, 4232, 4232, 4232, 4209, 4232, 4209, 4232, 4232, 4232, 4232, 4231, 4209, 4232, 4232, 4209, 4209, 4232, 4232, 4232, 4232, 4231, 4232, 4209, 4232, 4232, 4232, 4231, 4232, 4232, 4209, 4232, 4232, 4209, 4232, 4608, 4232, 4209, 4232, 4232, 4232, 4232, 4209, 4608, 4209, 4209, 4232, 4209, 4232, 4232, 4648, 4232, 4232, 4648, 4232, 4648, 4232, 4648, 4648, 4232, 4648, 4648, 4648, 4232, 4648, 4648, 4232, 4232, 4648, 4232, 4232, 4648, 4648, 4648, 4648, 4232, 4648, 4232, 4648, 4232, 4648, 4232, 4232, 4232, 4648, 4648, 4232, 4232, 4648, 4232, 4648, 4648, 4232, 4648, 4648, 4648, 4648, 4648, 4648, 4648, 4648, 4654, 4654, 4654, 4654, 4654, 4654, 4654, 4685, 4685, 4654, 4654, 4685, 4685, 4654, 4685, 4654, 4685, 4685, 4685, 4654, 4654, 4685, 4654, 4654, 4685, 4654, 4685, 4654, 4656, 4685, 4654, 4654, 4685, 4656, 4654, 4654, 4654, 4685, 4654, 4685, 4656, 4654, 4685, 4654, 4654, 4654, 4685, 4654, 4654, 4654, 4685, 4654, 4656, 4685, 4654, 4654, 4685, 4654, 4685, 4656, 4654, 4685, 4654, 4654, 4653, 4685, 4653, 4654, 4685, 4654, 4685, 4653, 4654, 4685, 4685, 4685, 4654, 4656, 4656, 4653, 4654, 4685, 4653, 4653, 4654, 4653, 4654, 4685, 4653, 4656, 4685, 4654, 4656, 4663, 4685, 4654, 4685, 4685, 4653, 4685, 4685, 4663, 4685, 4685, 4656, 4653, 4685, 4685, 4654, 4685, 4685, 4654, 4685, 4663, 4685, 4685, 4654, 4663, 4685, 4663, 4685, 4653, 4653, 4656, 4685, 4685, 4685, 4654, 4685, 4663, 4685, 4685, 4654, 4684, 4653, 4653, 4654, 4685, 4684, 4685, 4656, 4685, 4654, 4685, 4663, 4658, 4656, 4663, 4663, 4658, 4663, 4654, 4653, 4663, 4653, 4684, 4685, 4685, 4663, 4685, 4654, 4656, 4654, 4653, 4654, 4685, 4685, 4653, 4684, 4684, 4663, 4654, 4685, 4654, 4685, 4654, 4658, 4653, 4654, 4656, 4688, 4654, 4656, 4688, 4653, 4685, 4685, 4685, 4685, 4685, 4688, 4654, 4684, 4685, 4685, 4685, 4654, 4688, 4685, 4654, 4656, 4654, 4685, 4688, 4654, 4684, 4685, 4663, 4685, 4656, 4653, 4688, 4685, 4653, 4688, 4658, 4684, 4654, 4654, 4688, 4685, 4654, 4685, 4654, 4658, 4688, 4685, 4685, 4688, 4685, 4654, 4685, 4654, 4685, 4653, 4688, 4685, 4685, 4684, 4685, 4654, 4685, 4688, 4654, 4688, 4684, 4658, 4688, 4688, 4688, 4685, 4688, 4685, 4688, 4685, 4688, 4654, 4685, 4685, 4685, 4685, 4688, 4685, 4685, 4660, 4685, 4688, 4688, 4654, 4654, 4685, 4688, 4685, 4653, 4685, 4685, 4910, 4688, 4688, 4685, 4685, 4685, 4685, 4688, 4910, 4685, 4685, 4688, 4910, 4703, 4685, 4703, 4910, 4703, 4685, 4752, 4703, 4703, 4910, 4703, 4910, 4910, 4829, 4910, 4765, 4910, 4910, 4829, 4837, 4801, 4829, 4910, 4910, 4910, 4910, 4822, 4838, 4910, 4858, 4910, 4910, 4910, 4910, 4910, 4910, 4910, 4910, 4910, 4910, 4910, 4910, 4910, 4910, 4910, 4960, 4910, 4910, 4910, 4910, 4910, 4960, 4910, 4910, 4960, 4910, 4910, 4910, 4910, 4970, 4910, 4982, 4960, 4960, 4981, 5031, 5046, 5064, 5064, 4910, 4910, 5051, 4910, 4910, 4910, 4910, 5032, 5032, 5060, 5071, 5072, 4910, 4910, 4910, 5103, 4910, 4910, 5032, 4910, 4910, 5032, 5123, 5072, 5107, 4910, 4910, 5032, 4910, 5211, 4910, 5211, 5369, 5211, 5369, 5211, 5211, 5369, 5369, 5369, 5369, 5369, 5369, 5170, 5369, 5369, 5369, 5369, 5150, 5369, 5257, 5369, 5369, 5211, 5211, 5211, 5369, 5257, 5257, 5369, 5369, 5211, 5369, 5369, 5369, 5369, 5369, 5277, 5277, 5369, 5257, 5369, 5369, 5369, 5257, 5369, 5369, 5369, 5369, 5322, 5347, 5257, 5369, 5369, 5369, 5322, 5369, 5369, 5369, 5369, 5369, 5369, 5369, 5369, 5257, 5257, 5375, 5369, 5369, 5369, 5369, 5369, 5322, 5369, 5369, 5368, 5369, 5369, 5257, 5369, 5369, 5376, 5369, 5369, 5410, 5369, 5369, 5257, 5368, 5369, 5378, 5390, 5369, 5369, 5472, 5472, 5472, 5472, 5472, 5472, 5472, 5472, 5472, 5472, 5472, 5472, 5373, 5373, 5472, 5447, 5472, 5472, 5472, 5373, 5373, 5461, 5472, 5322, 5472, 5472, 5472, 5472, 5472, 5472, 5368, 5472, 5472, 5472, 5368, 5472, 5488, 5472, 5487, 5472, 5487, 5472, 5572, 5487, 5472, 5572, 5572, 5572, 5368, 5572, 5572, 5542, 5368, 5512, 5572, 5368, 5542, 5572, 5572, 5542, 5368, 5572, 5572, 5527, 5572, 5572, 5572, 5572, 5572, 5368, 5368, 5542, 5542, 5542, 5572, 5572, 5572, 5368, 5572, 5542, 5572, 5572, 5373, 5572, 5542, 5572, 5542, 5572, 5572, 5572, 5572, 5572, 5572, 5572, 5373, 5572, 5368, 5572, 5572, 5575, 5572, 5373, 5572, 5572, 5572, 5578, 5572, 5572, 5579, 5572, 5572, 5572, 5578, 5572, 5572, 5572, 5572, 5542, 5572, 5572, 5572, 5572, 5572, 5368, 5572, 5572, 5572, 5572, 5572, 5586, 5572, 5586, 5368, 5572, 5572, 5594, 5572, 5610, 5610, 5626, 5572, 5609, 5572, 5616, 5368, 5626, 5616, 5610, 5572, 5572, 5650, 5623, 5625, 5650, 5572, 5626, 5650, 5650, 5650, 5610, 5752, 5752, 5650, 5650, 5752, 5645, 5650, 5650, 5625, 5650, 5646, 5625, 5650, 5752, 5650, 5650, 5650, 5752, 5723, 5651, 5650, 5650, 5650, 5650, 5655, 5682, 5752, 5650, 5650, 5723, 5752, 5752, 5752, 5752, 5752, 5752, 5660, 5650, 5752, 5650, 5752, 5752, 5650, 5650, 5650, 5752, 5752, 5752, 5752, 5672, 5752, 5752, 5682, 5752, 5752, 5723, 5752, 5752, 5752, 5686, 5752, 5686, 5752, 5723, 5723, 5752, 5699, 5752, 5752, 5752, 5752, 5699, 5752, 5752, 5752, 5752, 5723, 5752, 5752, 5752, 5722, 5752, 5723, 5728, 5728, 5752, 5752, 5752, 5752, 5752, 5752, 5723, 5752, 5752, 5752, 5752, 5752, 5752, 5732, 5752, 5723, 5723, 5752, 5752, 5723, 5752, 5752, 5752, 5752, 5765, 5752, 5752, 5781, 5752, 5768, 5723, 5752, 5752, 5752, 5752, 5752, 5752, 5752, 5796, 5752, 5752, 5798, 5752, 5752, 5752, 5752, 5818, 5723, 5752, 5752, 5752, 5843, 5752, 5843, 5752, 5810, 5818, 5843, 5752, 5843, 5752, 5818, 5843, 5843, 5752, 5752, 29, 5843, 5752, 5752, 5843, 5752, 5843, 5843, 5828, 5829, 5843, 29, 5752, 29, 5843, 29, 5843, 5752, 5843, 5843, 29, 5843, 5843, 29, 5843, 5857, 5843, 5843, 5843]
idx = np.array(idx)

## in_crossmatch array (len = 4195)
in_crossmatch_ACT = [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, True, False, False, False, True, False, False, False, False, True, False, False, False, False, False, True, True, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, True, False, False, False, False, False, True, True, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, True, False, True, False, False, False, False, False, False, False, False, False, False, False, False, True, True, False, True, False, False, False, False, False, False, False, False, True, False, True, False, False, True, False, False, False, False, False, False, False, False, False, True, False, True, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, True, False, True, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, True, True, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, True, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, True, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, True, False, False, False, True, False, True, False, True, False, False, True, True, False, True, False, False, False, True, False, False, True, False, False, False, False, False, False, False, True, False, True, True, False, False, True, False, True, False, False, False, True, False, False, False, True, False, True, True, False, False, True, False, True, False, False, False, False, False, True, True, False, False, False, True, False, False, False, False, False, False, False, False, False, True, True, True, False, False, False, False, False, False, False, True, False, True, False, False, True, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, True, False, False, False, False, False, False, False, False, False, False, True, False, False, False, True, True, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, True, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, True, False, True, True, False, True, False, False, False, False, False, False, False, False, False, False, False, True, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, True, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, True, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, True, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, True, True, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, True, False, False, False, True, False, False, False, True, True, False, False, False, False, False, True, False, True, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, True, False, False, False, False, False, True, False, False, False, False, True, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, True, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, True, True, False, False, False, False, False, False, False, True, True, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, True, True, False, True, False, False, False, False, False, False, False, False, False, False, False, True, False, True, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, True, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, True, False, False, False, True, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, True, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, True, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False]
in_crossmatch_ACT = np.array(in_crossmatch_ACT)

## in_crossmatch (indexes into a 5860-element HSC array)
in_crossmatch_HSC = idx[in_crossmatch_ACT]

internal_cat_IDs[in_crossmatch_HSC]

array(['C19a1sW00048', 'C19a1sW00101', 'C19a1sW00140', 'C19a1sW00166',
       'C19a1sW00191', 'C19a1sW00200', 'C19a1sW00226', 'C19a1sW00302',
       'C19a1sW00347', 'C19a1sW00364', 'C19a1sW00368', 'C19a1sW00373',
       'C19a1sW00450', 'C19a1sW00510', 'C19a1sW00519', 'C19a1sW00568',
       'C19a1sW00577', 'C19a1sW00597', 'C19a1sW00638', 'C19a1sW00645',
       'C19a1sW00671', 'C19a1sW00723', 'C19a1sW00730', 'C19a1sW00825',
       'C19a1sW00860', 'C19a1sW00866', 'C19a1sW00878', 'C19a1sW00955',
       'C19a1sW01001', 'C19a1sW01015', 'C19a1sW01054', 'C19a1sW01156',
       'C19a1sW01237', 'C19a1sW01266', 'C19a1sW01272', 'C19a1sW01400',
       'C19a1sW01470', 'C19a1sW01515', 'C19a1sW01583', 'C19a1sW01596',
       'C19a1sW01639', 'C19a1sW01676', 'C19a1sW01711', 'C19a1sW01713',
       'C19a1sW01953', 'C19a1sW02098', 'C19a1sW02192', 'C19a1sW02209',
       'C19a1sW02224', 'C19a1sW02332', 'C19a1sW02334', 'C19a1sW02384',
       'C19a1sW02468', 'C19a1sW02498', 'C19a1sW02585', 'C19a1sW02605',
      

In [88]:
public_cat_IDs[in_crossmatch_HSC]

array(['HSCJ000217+013157', 'HSCJ000350+020359', 'HSCJ000500+021143',
       'HSCJ000547+022256', 'HSCJ000634+001806', 'HSCJ000654-004120',
       'HSCJ000810+020112', 'HSCJ015654-042426', 'HSCJ015826-014639',
       'HSCJ015913-032420', 'HSCJ015921-030955', 'HSCJ015931-035023',
       'HSCJ020147-021154', 'HSCJ020300-004236', 'HSCJ020317-012401',
       'HSCJ020447-011713', 'HSCJ020456-030321', 'HSCJ020517-043920',
       'HSCJ020613-011500', 'HSCJ020623-011832', 'HSCJ020658-011951',
       'HSCJ020816-023724', 'HSCJ020820-025608', 'HSCJ021002-024411',
       'HSCJ021056-061154', 'HSCJ021102-045318', 'HSCJ021115-034324',
       'HSCJ021321-060437', 'HSCJ021426-062740', 'HSCJ021441-043402',
       'HSCJ021528-044041', 'HSCJ021835-011434', 'HSCJ022056-033348',
       'HSCJ022146-034619', 'HSCJ022156-034054', 'HSCJ022528-035502',
       'HSCJ022738-031757', 'HSCJ022936-033623', 'HSCJ023141-045257',
       'HSCJ023217-061914', 'HSCJ023336-053022', 'HSCJ023516-032348',
       'HSCJ023643-0

Then, use in_crossmatch_HSC to index into the public list:

In [87]:
ind = 43
print('Public name: ' + public_cat_IDs[in_crossmatch_HSC][ind])
print(f'Internal RA coord (HMS): {Angle(internal_ra[in_crossmatch_HSC][ind], u.rad).hms}')
print(f'Internal Dec coord (DMS): {Angle(internal_dec[in_crossmatch_HSC][ind], u.rad).dms}')

Public name: HSCJ023655-060708
Internal RA coord (HMS): hms_tuple(h=2.0, m=36.0, s=54.94656000000475)
Internal Dec coord (DMS): dms_tuple(d=-6.0, m=-7.0, s=-7.752000000001331)


Sanity check: the name lists RA in HMS notation and dec in deg/arcmin/arcsec. So can we replicate the names using the internal catalog's coordinates?